In [1]:
import pandas as pd
import numpy as np
from keras.utils.np_utils import to_categorical 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator

/home/kriti/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load Data
digit_train = pd.read_csv('train.csv')
digit_test = pd.read_csv('test.csv')

In [3]:
#split into X and Y
train_X = digit_train.drop("label",axis = 1)
train_Y = digit_train["label"]

In [4]:
#Normalization
train_X = train_X / 255.0
test = digit_test / 255.0

In [5]:
#Reshape
train_X = train_X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [6]:
train_X.shape

(42000, 28, 28, 1)

In [7]:
#Label Encoder
train_Y = to_categorical(train_Y, num_classes = 10)

In [8]:
random_seed = 2

In [9]:
train_X, X_val, train_Y, Y_val = train_test_split(train_X, train_Y, test_size = 0.1, random_state=random_seed)

In [10]:
mean_px = train_X.mean().astype(np.float32)
std_px = train_X.std().astype(np.float32)

def standardize(x): 
    return (x-mean_px)/std_px

In [11]:
from keras.models import  Sequential
from keras.layers.core import  Lambda , Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D

In [12]:
model= Sequential()
model.add(Lambda(standardize,input_shape=(28,28,1)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
print("input shape ",model.input_shape)
print("output shape ",model.output_shape)

input shape  (None, 28, 28, 1)
output shape  (None, 10)


In [13]:
from keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001),
 loss='categorical_crossentropy',
 metrics=['accuracy'])


In [14]:
from keras.preprocessing import image
gen = image.ImageDataGenerator()

In [15]:
from sklearn.model_selection import train_test_split
X = train_X
y = train_Y
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)
batches = gen.flow(train_X, train_Y, batch_size=64)
val_batches=gen.flow(X_val, Y_val, batch_size=64)

In [16]:
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=3, 
                    validation_data=val_batches, validation_steps=val_batches.n)

Epoch 1/3
37800/37800 [==============================] - 86s 2ms/step - loss: 0.2413 - acc: 0.9333 - val_loss: 0.3240 - val_acc: 0.9152
Epoch 2/3
37800/37800 [==============================] - 86s 2ms/step - loss: 0.2170 - acc: 0.9410 - val_loss: 0.3415 - val_acc: 0.9145
Epoch 3/3
37800/37800 [==============================] - 80s 2ms/step - loss: 0.2113 - acc: 0.9430 - val_loss: 0.3820 - val_acc: 0.9081


In [17]:
from keras.layers import Convolution2D, MaxPooling2D

def get_cnn_model():
    model = Sequential([
        Lambda(standardize, input_shape=(28,28,1)),
        Convolution2D(32,(3,3), activation='relu'),
        Convolution2D(32,(3,3), activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,(3,3), activation='relu'),
        Convolution2D(64,(3,3), activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [18]:
model= get_cnn_model()
model.optimizer.lr=0.01

In [19]:
history=model.fit_generator(generator=batches, steps_per_epoch=batches.n, epochs=1, 
                    validation_data=val_batches, validation_steps=val_batches.n)


Epoch 1/1
37800/37800 [==============================] - 3074s 81ms/step - loss: 14.5085 - acc: 0.0998 - val_loss: 14.5715 - val_acc: 0.0960


In [20]:
predictions = model.predict_classes(test, verbose=0)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
                         "Label": predictions})
submissions.to_csv("result.csv", index=False)